In [ ]:
# !curl -X GET "nersc-tbn-6.testbed100.es.net:9200/iperf3*/_search" | jq 

In [13]:
'''
**
** Project Lead: Eashan Adhikarla
** Mentor: Ezra Kissel
** 
** Date Created: June 17' 2021
** Last Modified: June 22' 2021 
**
'''

from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

import pandas as pd
import requests
import json

# Create the elasticsearch client
HOST = 'nersc-tbn-6.testbed100.es.net'
PORT = 9200

es = Elasticsearch(host=HOST, port=PORT)


class clr:
    """
    Defining colors for the print syntax coloring
    """
    H   = '\033[35m' # Header
    B   = '\033[94m' # Blue
    G   = '\033[36m' # Green
    W   = '\033[93m' # Warning
    F   = '\033[91m' # Fail
    E   = '\033[0m'  # End 
    BD  = '\033[1m'  # Bold
    UL  = '\033[4m'  # Underline


class GETTER:
    """
    Get class to get current information about different Indexs
    """
    def __init__(self, term, sum=0):
        self.term = term
        self.sum = sum

    def getIndexList(self, term):
        idx=[]
        indices_dict = es.indices.get_alias(self.term)
        if isinstance(indices_dict, dict) and indices_dict is not None:
            print(f"'{clr.BD}{clr.G}{len(indices_dict)}{clr.E}{clr.E}' indexes found!")
            for k,v in indices_dict.items():
                idx.append(k)
            return idx
        else:
            print (f"{clr.F}Empty dict!{clr.E}")

    def getIndexedDetails(self, indexes):
        for i in range(len(indexes)):
            result = es.search(index=indexes[i], 
                            body={"query":{"match_all":{}}}, 
                            size=10000,
                            )
            print(f"Found '{clr.BD}{clr.G}{result['hits']['total']['value']}{clr.E}{clr.E}' documents in the current index '{clr.BD}{clr.G}{indexes[i]}{clr.E}{clr.E}'")
            
            data = [doc for doc in result['hits']['hits']]

            self.sum += result['hits']['total']['value']
        print("Total docs found: ", self.sum)
            # # iterate the nested dictionaries inside the ["hits"]["hits"] list
            # for num, doc in enumerate(data):
            #     # print ("DOC ID:", doc["_id"], "--->", doc, type(doc), "\n")
                
            #     # Use 'iteritems()` instead of 'items()' if using Python 2
            #     for key, value in doc.items():
            #         print (f"{clr.B}{key}{clr.E}   ---> {value}")

            #     # print a few spaces between each doc for readability
            #     print ("\n\n")
            #     break



indexTypes = ["*", "iperf*", "jobmeta*", "bbrmon*"]
term_ = indexTypes[1]

get = GETTER(term_)
indexes = get.getIndexList(term_)
for e,i in enumerate(indexes):
    print(f"{e}: {i}")

index_response = get.getIndexedDetails(indexes)



'4' indexes found!
0: iperf3-2021.06.22
1: iperf3-2021.06.04
2: iperf3-2021.06.06
3: iperf3-2021.06.05
Found '48' documents in the current index 'iperf3-2021.06.22'
Found '10' documents in the current index 'iperf3-2021.06.04'
Found '2' documents in the current index 'iperf3-2021.06.06'
Found '2' documents in the current index 'iperf3-2021.06.05'
Total docs found:  62


In [ ]:
# es.get(index="iperf3-2021.06.06", id="uhKs33kBkImc33Nl6K8P")

In [ ]:
# result = es.search(index=indexes[0], body={"query":{"match_all":{}}})
# indexes[0]

In [ ]:
result = es.search(index=indexes[0], body={"query":{"match_all":{}}})

# hitsList=[]
# print(f"Found {len(result.items())} keys found..\n")

print(f"Found '{clr.B}{clr.BD}{result['hits']['total']['value']}{clr.E}{clr.E}' documents in the current index'{clr.B}{clr.BD}{indexes[0]}{clr.E}{clr.E}'")

data = [doc for doc in result['hits']['hits']]

i=0
print(data[i]['_source'].keys())
print("\n")
print(data[i]['_source']['end'])
# print(data[i]['_source']['@version'])
# print(data[i]['_source']['start'])
# print(data[i]['_source']['uuid'])
# print(data[i]['_source']['@timestamp'])
print()

i=1
print(data[i]['_source'].keys())
print("\n")
print(data[i]['_source']['end'])
# print(data[i]['_source']['@version'])
# print(data[i]['_source']['start'])
# print(data[i]['_source']['uuid'])
# print(data[i]['_source']['@timestamp'])
print()

i=2
print(data[i]['_source'].keys())
print("\n")
print(data[i]['_source']['end'])
# print(data[i]['_source']['@version'])
# print(data[i]['_source']['start'])
# print(data[i]['_source']['uuid'])
# print(data[i]['_source']['@timestamp'])
print()

# for doc in data:
#     print(doc['_source'].keys())
#     print(doc['_source']['start'])
#     print(doc['_source']['uuid'])
#     print(doc['_source']['@timestamp'])
#     break
    # print(f"{doc['_id']}, {doc['_source']}"), {doc['_source']}" )

In [ ]:
for k2,v2 in hitsList[0][0].items():
    print(k2)
    if k2=="_score":
        for v in value:
            print(v)

In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch(host='nersc-tbn-6.testbed100.es.net', port=9200)
res = es.search(index="iperf3*",
                # doc_type="articles",
                body={"query": 
                        {"match": 
                                {"content": ""}
                        }
                      })
print("res: ", res)

print(f"{res['hits']['total']} documents found")

for doc in res['hits']['hits']:
    print("%s) %s" % (doc['_id'], doc['_source']['content']))


## Querying elasticsearch via elasticsearch-py module

In [ ]:
# def get_data_from_elastic():
    
# query: The elasticsearch query.
query = {
    "query": {
        "match": {
            "symbols.keyword": ""
        }
    }
}

# Scan function to get all the data. 
rel = scan( client=es,             
            query=query,                                     
            scroll='1m',
            index='iperf3*',
            raise_on_error=True,
            preserve_order=True,
            clear_scroll=True)

# Keep response in a list.
if not rel==None:
    result = list(rel)

print(result)

# temp = []

# # We need only '_source', which has all the fields required.
# # This elimantes the elasticsearch metdata like _id, _type, _index.
# for hit in result:
#     temp.append(hit['_source'])

# # Create a dataframe.
# df = pd.DataFrame(temp)

# return df


## Querying elasticsearch via REST api

In [ ]:
def search(uri, term):
    """
    Simple Elasticsearch Query
    """
    query = json.dumps({
        "query": {
            "match": {
                "content": term
            }
        }
    })
    response = requests.get(uri, data=query)
    results = json.loads(response.text)
    return results

def format_results(results):
    """
    Print results nicely:
    doc_id) content
    """
    data = [doc for doc in results['hits']['hits']]
    for doc in data:
        print("%s) %s" % (doc['_id'], doc['_source']['content']))

def create_doc(uri, doc_data={}):
    """
    Create new document.
    """
    query = json.dumps(doc_data)
    response = requests.post(uri, data=query)
    print(response)

In [ ]:
# df = get_data_from_elastic()
# print(df.head())

In [ ]:
if __name__ == '__main__':
    uri_search = 'http://192.168.120.46:9200/'
    # uri_search = 'http://192.168.120.46:9200/test/articles/_search'
    # uri_create = 'http://localhost:9200/test/articles/'

    results = search(uri_search, "_source")
    print(results)
    # format_results(results)

    #create_doc(uri_create, {"content": "The fox!"})
    #results = search(uri_search, "fox")
    #format_results(results)

In [ ]:
# !curl -XGET 192.168.120.46:9200/iperf3*/_search | jq >> sample.txt

In [ ]:
import subprocess

list_files = subprocess.run(["ls", "-al"])
print("The exit code was: %d" % list_files.returncode)